In [66]:
import pandas as pd
import numpy as np
import seaborn as sns

import requests
import json
import geopy

import time
import parquet


In [44]:
data = pd.read_csv('despesas-entre-2003-e-2022.csv', encoding='latin1', sep=';', dtype='str')

In [45]:
data['SUBELEMENTO DE DESPESA'] = data['SUBELEMENTO DE DESPESA'].astype('category')

In [59]:
data

,DATA PGTO,CPF SERVIDOR,CPF/CNPJ FORNECEDOR,NOME FORNECEDOR,VALOR,TIPO,SUBELEMENTO DE DESPESA,CDIC,ATIVIDADE_PRINCIPAL,MATRIZ,NATUREZA_JURIDICA,ENDEREÇO,LOGRADOURO,NUMERO,COMPLEMENTO,BAIRRO,MUNICIPIO,UF,CEP
0,02/01/2003,***.010.008-**,31349202000177,JW TRANSPORTADORA TURÖSTICA,"R$ 1.696,90",D,LOCACAO DE MEIOS DE TRANSPORTE,00200.220539/2003-63.R.08.18/12/2002.31/12/2014.N,Comércio varejista de plantas e flores naturais,MATRIZ,206-2 - Sociedade Empresária Limitada,"SHC/SUL CL QUADRA 207 BLOCO B LOJA, 17 - - AS...",SHC/SUL CL QUADRA 207 BLOCO B LOJA,17,,ASA SUL,BRASILIA,DF,70.253-520
1,02/01/2003,***.010.008-**,31349202000177,JW TRANSPORTADORA TURÖSTICA,"R$ 1.336,60",D,LOCACAO DE MEIOS DE TRANSPORTE,00200.220539/2003-63.R.08.18/12/2002.31/12/2014.N,,,,NaN,,,,,,,
2,02/01/2003,***.868.251-**,31349202000177,JW TRANSPORTADORA TURÖSTICA,"R$ 1.396,43",D,LOCACAO DE MEIOS DE TRANSPORTE,00200.600539/2003-63.R.08.18/12/2002.31/12/2014.N,,,,NaN,,,,,,,
3,03/01/2003,***.004.131-**,00592717000170,FLORES ALVORADA,"R$ 8.585,00",C,OUTROS SERVIOS DE TERCEIROS-PESSOA JURÖDICA,00200.800212/2003-91.R.08.07/12/2002.31/12/2014.N,,,,NaN,,,,,,,
4,03/01/2003,***.004.131-**,00839308000125,CRISTAL LIMPEZA - COMRCIO DE MATERIAIS PARA L...,"R$ 251,50",C,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZAÇO,00200.800212/2003-91.R.08.07/12/2002.31/12/2014.N,,,,NaN,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113337,04/12/2022,***.136.591-**,47508411094975,COMPANHIA BRASILEIRA DE DISTRIBUIÇO,"518,07",C,GENEROS DE ALIMENTAÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,,,,NaN,,,,,,,
113338,04/12/2022,***.136.591-**,12550441000146,RESTAURANTE SOHO,"330,52",C,FORNECIMENTO DE ALIMENTAÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,,,,NaN,,,,,,,
113339,04/12/2022,***.136.591-**,03696869000100,BIG TRANS COMERCIAL DE ALIMENTOS LTDA,"54,66",C,GENEROS DE ALIMENTAÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,,,,NaN,,,,,,,
113340,Fonte: SUPRIM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,NaN,,,,,,,


In [47]:
def get_company_info(cnpj):
    url = f"https://receitaws.com.br/v1/cnpj/{cnpj}"
    querystring = {"token":"XXXXXXXX-XXXX-XXXX-XXXX-XXXXXXXXXXXX","cnpj":"06990590000123","plugin":"RF"}
    response = requests.request("GET", url, params=querystring)
    return json.loads(response.text)

def print_company_adress(info):
    print(f"{info['nome']}: {info['logradouro']}, {info['numero']} ({info['complemento']}) - CEP {info['cep']}")


info = get_company_info(GOOGLE_CNPJ)
print_company_adress(info)


BIG TRANS COMERCIAL DE ALIMENTOS S/A: SHCN EQ 402/403, SN (BLOCO A) - CEP 70.834-400


In [48]:
GOOGLE_CNPJ = "03696869000100"
info = get_company_info(GOOGLE_CNPJ)
print(info)
#atividade_principal

#tipo

#logradouro
#número
#bairro
#municipio
#uf
#cep

#natureza_juridica


{'atividade_principal': [{'code': '47.11-3-02', 'text': 'Comércio varejista de mercadorias em geral, com predominância de produtos alimentícios - supermercados'}], 'data_situacao': '03/11/2005', 'complemento': 'BLOCO A', 'tipo': 'MATRIZ', 'nome': 'BIG TRANS COMERCIAL DE ALIMENTOS S/A', 'abertura': '04/11/1999', 'telefone': '(61) 3265-191', 'qsa': [{'nome': 'JORGE HELOU FILHO', 'qual': '10-Diretor'}, {'nome': 'MARIO HABKA', 'qual': '10-Diretor'}], 'situacao': 'ATIVA', 'bairro': 'ASA NORTE', 'logradouro': 'SHCN EQ 402/403', 'numero': 'SN', 'cep': '70.834-400', 'municipio': 'BRASILIA', 'uf': 'DF', 'porte': 'DEMAIS', 'natureza_juridica': '205-4 - Sociedade Anônima Fechada', 'fantasia': 'BIG BOX SUPERMERCADOS', 'cnpj': '03.696.869/0001-00', 'ultima_atualizacao': '2022-12-27T17:17:28.815Z', 'status': 'OK', 'email': '', 'efr': '', 'motivo_situacao': '', 'situacao_especial': '', 'data_situacao_especial': '', 'atividades_secundarias': [{'code': '00.00-0-00', 'text': 'Não informada'}], 'capital_

In [64]:
info = get_company_info(data['CPF/CNPJ FORNECEDOR'].iloc[1])



In [65]:
info

{'atividade_principal': [{'text': 'Transporte rodoviário coletivo de passageiros, sob regime de fretamento, intermunicipal, interestadual e internacional',
   'code': '49.29-9-02'}],
 'data_situacao': '07/08/2004',
 'tipo': 'MATRIZ',
 'nome': 'JW TRANSPORTADORA TURISTICA LTDA',
 'abertura': '03/02/1987',
 'telefone': '(21) 3553-9170/ (21) 3885-0085',
 'email': 'administrativo@jwtransportes.com.br',
 'atividades_secundarias': [{'text': 'Serviço de transporte de passageiros - locação de automóveis com motorista',
   'code': '49.23-0-02'},
  {'text': 'Locação de automóveis sem condutor', 'code': '77.11-0-00'}],
 'qsa': [{'nome': 'ALEXANDRE PINHEIRO DE ALBUQUERQUE', 'qual': '22-Sócio'},
  {'nome': 'RODRIGO VILLELA GOULART', 'qual': '49-Sócio-Administrador'},
  {'nome': 'JOSE FERNANDO BATISTA DE ALBUQUERQUE', 'qual': '22-Sócio'}],
 'situacao': 'ATIVA',
 'bairro': 'BONSUCESSO',
 'logradouro': 'R BARROS BARRETO',
 'numero': '74',
 'cep': '21.032-140',
 'municipio': 'RIO DE JANEIRO',
 'porte':

In [95]:

data['ATIVIDADE_PRINCIPAL'] = ''
data['MATRIZ'] = ''
data['NATUREZA_JURIDICA'] = ''
data['LOGRADOURO'] = ''
data['NUMERO'] = ''
data['COMPLEMENTO'] = ''
data['BAIRRO'] = ''
data['MUNICIPIO'] = ''
data['UF'] = ''
data['CEP'] = ''

for i in range(1000):
    try:
        info = get_company_info(data['CPF/CNPJ FORNECEDOR'].iloc[i])
        data.loc[i,'ATIVIDADE_PRINCIPAL'] = info['atividade_principal'][0]['text']
        data.loc[i,'MATRIZ'] = info['tipo']
        data.loc[i,'NATUREZA_JURIDICA'] = info['natureza_juridica']
        data.loc[i,'LOGRADOURO'] = info['logradouro'] 
        data.loc[i,'NUMERO'] = info['numero'] 
        data.loc[i,'COMPLEMENTO'] = info['complemento'] 
        data.loc[i,'BAIRRO'] = info['bairro'] 
        data.loc[i,'MUNICIPIO'] = info['municipio'] 
        data.loc[i,'UF'] = info['uf'] 
        data.loc[i,'CEP'] = info['cep'] 
        data.loc[i,'ENDEREÇO'] = info['logradouro'] + ', '+ info['numero']+', '+ info['municipio'] +', ' + info['uf']
    except:
        print('Erro:',i)
    time.sleep(20)




KeyboardInterrupt: 

In [96]:
data.to_parquet('Dados_processador_metade.parquet')

In [97]:
pd.read_parquet('Dados_processador_metade.parquet')

,DATA PGTO,CPF SERVIDOR,CPF/CNPJ FORNECEDOR,NOME FORNECEDOR,VALOR,TIPO,SUBELEMENTO DE DESPESA,CDIC,ATIVIDADE_PRINCIPAL,MATRIZ,NATUREZA_JURIDICA,ENDEREÇO,LOGRADOURO,NUMERO,COMPLEMENTO,BAIRRO,MUNICIPIO,UF,CEP
0,02/01/2003,***.010.008-**,31349202000177,JW TRANSPORTADORA TURÖSTICA,"R$ 1.696,90",D,LOCACAO DE MEIOS DE TRANSPORTE,00200.220539/2003-63.R.08.18/12/2002.31/12/2014.N,"Transporte rodoviário coletivo de passageiros,...",MATRIZ,206-2 - Sociedade Empresária Limitada,"R BARROS BARRETO, 74, RIO DE JANEIRO, RJ",R BARROS BARRETO,74,,BONSUCESSO,RIO DE JANEIRO,RJ,21.032-140
1,02/01/2003,***.010.008-**,31349202000177,JW TRANSPORTADORA TURÖSTICA,"R$ 1.336,60",D,LOCACAO DE MEIOS DE TRANSPORTE,00200.220539/2003-63.R.08.18/12/2002.31/12/2014.N,"Transporte rodoviário coletivo de passageiros,...",MATRIZ,206-2 - Sociedade Empresária Limitada,"R BARROS BARRETO, 74, RIO DE JANEIRO, RJ",R BARROS BARRETO,74,,BONSUCESSO,RIO DE JANEIRO,RJ,21.032-140
2,02/01/2003,***.868.251-**,31349202000177,JW TRANSPORTADORA TURÖSTICA,"R$ 1.396,43",D,LOCACAO DE MEIOS DE TRANSPORTE,00200.600539/2003-63.R.08.18/12/2002.31/12/2014.N,"Transporte rodoviário coletivo de passageiros,...",MATRIZ,206-2 - Sociedade Empresária Limitada,"R BARROS BARRETO, 74, RIO DE JANEIRO, RJ",R BARROS BARRETO,74,,BONSUCESSO,RIO DE JANEIRO,RJ,21.032-140
3,03/01/2003,***.004.131-**,00592717000170,FLORES ALVORADA,"R$ 8.585,00",C,OUTROS SERVIOS DE TERCEIROS-PESSOA JURÖDICA,00200.800212/2003-91.R.08.07/12/2002.31/12/2014.N,Comércio varejista de plantas e flores naturais,MATRIZ,206-2 - Sociedade Empresária Limitada,"SHC/SUL CL QUADRA 207 BLOCO B LOJA, 17, BRASIL...",SHC/SUL CL QUADRA 207 BLOCO B LOJA,17,,ASA SUL,BRASILIA,DF,70.253-520
4,03/01/2003,***.004.131-**,00839308000125,CRISTAL LIMPEZA - COMRCIO DE MATERIAIS PARA L...,"R$ 251,50",C,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZAÇO,00200.800212/2003-91.R.08.07/12/2002.31/12/2014.N,Comércio varejista de produtos saneantes domis...,MATRIZ,206-2 - Sociedade Empresária Limitada,"ST SHCS CL QD 404 BL C LJS 09 E 13, SN, BRASIL...",ST SHCS CL QD 404 BL C LJS 09 E 13,SN,,ASA SUL,BRASILIA,DF,70.238-030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113337,04/12/2022,***.136.591-**,47508411094975,COMPANHIA BRASILEIRA DE DISTRIBUIÇO,"518,07",C,GENEROS DE ALIMENTAÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,,,,None,,,,,,,
113338,04/12/2022,***.136.591-**,12550441000146,RESTAURANTE SOHO,"330,52",C,FORNECIMENTO DE ALIMENTAÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,,,,None,,,,,,,
113339,04/12/2022,***.136.591-**,03696869000100,BIG TRANS COMERCIAL DE ALIMENTOS LTDA,"54,66",C,GENEROS DE ALIMENTAÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,,,,None,,,,,,,
113340,Fonte: SUPRIM,None,None,None,None,None,NaN,None,,,,None,,,,,,,
